In [1]:
from nba_api.stats.static import teams
from nba_api.stats.endpoints import leaguegamefinder
import pandas as pd
import os.path
from os import path

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [ ]:
nba_teams = teams.get_teams()

# This portion retreives all the scratch data using NBA API

In [ ]:
def getData(filename, start_date, end_date):
    if (path.exists(filename) != True):
        
        team_id = nba_teams[0]['id']
        gamefinder = leaguegamefinder.LeagueGameFinder(date_from_nullable = start_date , date_to_nullable = end_date , team_id_nullable = team_id)
        games = gamefinder.get_data_frames()[0]
        games.to_csv(filename,index=False)
        
    if (path.exists(filename) == True):   
        
        old_df = pd.read_csv(filename)
        last_id = old_df['TEAM_ID'][len(old_df)-1]
        start_id = int(last_id) + 1
        
        while start_id <= 1610612766:
            old_df = pd.read_csv(filename)
            gamefinder = leaguegamefinder.LeagueGameFinder(date_from_nullable = start_date , date_to_nullable = end_date , team_id_nullable = start_id)
            games = gamefinder.get_data_frames()[0]
            new_df = old_df.append(games)
            new_df.to_csv(filename, index=False)
            start_id = start_id + 1
def error_handle(count, filename, start_date, end_date):
    try:
        print("-----try is running-----")
        # put csv name here
        getData(filename, start_date, end_date)
        count = 0
        
    except:
        if count < 25:
            print("-----exception handled-----", count)
            error_handle(count + 1,filename, start_date, end_date)
        else:
            print("-----max tries exceeded-----")        

# Finds the ten day rolling averages per team

In [ ]:
#name of csv to read, name of csv to write
def rolling_average_stats(r_filename, w_filename):  
    csv_df = pd.read_csv(r_filename)

    list_points = []
    list_team_points = []
    x = 1
    for team in nba_teams:
        team_df = csv_df[csv_df['TEAM_ID'] == team['id']]
        for col in team_df.columns[9:]:
            team_df['AV_'+ col] = team_df[col].rolling(window=10).mean()
            team_df['AV_'+ col] = team_df['AV_'+ col].shift(1) #add code here
        head = list(team_df.columns.values)
        if x == 1:
#             new_df = team_df
            team_df.to_csv(w_filename, header=head, index=False)
            x = x+1

        else:
#             new_df.append(team_df)
            team_df.to_csv(w_filename, mode='a', header=False, index=False)
rolling_average_stats('correct_dates_19-20.csv', 'ten_day-19-20.csv')

In [ ]:
#name of csv to read, name of csv to write
def most_recent_team_data(r_filename, w_filename, start_date, end_date):  
    
    error_handle(0,r_filename,start_date, end_date)
    csv_df = pd.read_csv(r_filename)
    csv_df = csv_df.sort_values(['TEAM_ABBREVIATION','GAME_DATE'] , ascending=[True, True])

    list_points = []
    list_team_points = []
    x = 1
    for team in nba_teams:
        team_df = csv_df[csv_df['TEAM_ID'] == team['id']]
        for col in team_df.columns[9:]:
            team_df['AV_'+ col] = team_df[col].rolling(window=10).mean()
            team_df['AV_'+ col] = team_df['AV_'+ col]
        head = list(team_df.columns.values)
        if x == 1:
            team_df.tail(1).to_csv(w_filename, header=head, index=False)
            x = x+1

        else:
            team_df.tail(1).to_csv(w_filename, mode='a', header=False, index=False)
most_recent_team_data('10-8-2020.csv', 'last_game_10-8-2020.csv', "10/16/2019", "10/08/2020")

# Combines teams that play eachother and sorts data

In [ ]:
def combine_team_games(df, keep_method='home'):
    '''Combine a TEAM_ID-GAME_ID unique table into rows by game. Slow.

        Parameters
        ----------
        df : Input DataFrame.
        keep_method : {'home', 'away', 'winner', 'loser', ``None``}, default 'home'
            - 'home' : Keep rows where TEAM_A is the home team.
            - 'away' : Keep rows where TEAM_A is the away team.
            - 'winner' : Keep rows where TEAM_A is the losing team.
            - 'loser' : Keep rows where TEAM_A is the winning team.
            - ``None`` : Keep all rows. Will result in an output DataFrame the same
                length as the input DataFrame.
                
        Returns
        -------
        result : DataFrame
    '''
    # Join every row to all others with the same game ID.
    joined = pd.merge(df, df, suffixes=['_A', '_B'],
                      on=['SEASON_ID', 'GAME_ID', 'GAME_DATE'])
    # Filter out any row that is joined to itself.
    result = joined[joined.TEAM_ID_A != joined.TEAM_ID_B]
    # Take action based on the keep_method flag.
    if keep_method is None:
        # Return all the rows.
        pass
    elif keep_method.lower() == 'home':
        # Keep rows where TEAM_A is the home team.
        result = result[result.MATCHUP_A.str.contains(' vs. ')]
    elif keep_method.lower() == 'away':
        # Keep rows where TEAM_A is the away team.
        result = result[result.MATCHUP_A.str.contains(' @ ')]
    elif keep_method.lower() == 'winner':
        result = result[result.WL_A == 'W']
    elif keep_method.lower() == 'loser':
        result = result[result.WL_A == 'L']
    else:
        raise ValueError(f'Invalid keep_method: {keep_method}')
    return result

In [ ]:
# uses combine function
count = 0
for row in attempt.iterrows():
    if (count == 0):
        catch = attempt.loc[attempt['GAME_ID'] == row[1]['GAME_ID']]
        catch = pd.DataFrame(catch)
        combine = combine_team_games(catch)
        combine.to_csv('combined_19-20.csv', index=False)
        count = count + 1
    else: 
        old_df = pd.read_csv('combined_19-20.csv')
        catch = attempt.loc[attempt['GAME_ID'] == row[1]['GAME_ID']]
        catch = pd.DataFrame(catch)
        combine = combine_team_games(catch)
        new_df = old_df.append(combine)
        new_df.to_csv('combined_19-20.csv', index=False)
#         count = count + 1
#     if (count == 5):
#         break

In [ ]:
clean = pd.read_csv('combined_19-20.csv')
# drops duplicates, sort by game date, and replace W with 1 and L with 0
cleaned = clean.drop_duplicates(subset='GAME_ID')
cleaned = cleaned.sort_values('GAME_DATE')
cleaned['WL_A'] = cleaned['WL_A'].replace(['W','L'],[1,0])
cleaned['WL_B'] = cleaned['WL_B'].replace(['W','L'],[1,0])
cleaned.to_csv('combined_CLEANED_2019-2020.csv', index=False)

# Calculates zscores for each team + fits data to linear regression and decision tree model

In [ ]:
data = pd.read_csv('combined_CLEANED_2019-2020.csv')
z_data = pd.DataFrame(columns = ['WL', 'PTS', 'FGM' , 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PLUS_MINUS'])
z_data['WL'] = data['WL_A']
for column in z_data.columns[1:]:
    z_data[column] = data['AV_' + column + '_A'] - data['AV_' + column + '_B']
z_data = z_data.dropna()
z_data.head()
z_data.to_csv('z_scores_19-20.csv')

In [ ]:
# Creates the logistic regression model and tests accuracy
def performLogReg(dataframe):

    # Update if new stats are added
    featureColumns = ['PTS', 'FGM', 'FGA', 'FG3_PCT', 'FTA','REB', 'AST',  'STL', 'TOV']

    X = dataframe[featureColumns] # Features
    Y = dataframe['WL'] # Target Variable

    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, shuffle=True)
    logreg = LogisticRegression()

    logreg.fit(X_train, Y_train)  # Fits model with data

    Y_pred = logreg.predict(X_test)

    confusionMatrix = metrics.confusion_matrix(Y_test, Y_pred)  # Diagonals tell you correct predictions

    # Code below prints model accuracy information
    print('Coefficient Information:')

    for i in range(len(featureColumns)):  # Prints each feature next to its corresponding coefficient in the model

        logregCoefficients = logreg.coef_

        currentFeature = featureColumns[i]
        currentCoefficient = logregCoefficients[0][i]

        print(currentFeature + ': ' + str(currentCoefficient))

    print('----------------------------------')

    print("Accuracy:", metrics.accuracy_score(Y_test, Y_pred))
    print("Precision:", metrics.precision_score(Y_test, Y_pred))
    print("Recall:", metrics.recall_score(Y_test, Y_pred))

    print('----------------------------------')

    print('Confusion Matrix:')
    print(confusionMatrix)

    return logreg

In [ ]:
df1 = pd.read_csv("z_scores_17-18.csv")
df2 = pd.read_csv("z_scores_18-19.csv")
df3 = pd.read_csv("z_scores_19-20.csv")
df1 = df1.append(df2)
df1 = df1.append(df3)
performLogReg(df1)

In [ ]:
def performDecTree(dataframe):
    # Update if new stats are added
    featureColumns = ['PTS', 'FGM', 'FGA', 'FG3_PCT', 'FTA','REB', 'AST',  'STL', 'TOV']

    X = dataframe[featureColumns] # Features
    Y = dataframe['WL'] # Target Variable
    
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=1)
    
    model = tree.DecisionTreeClassifier(max_depth=2)
    model.fit(X_train, Y_train)
    
    Y_predict = model.predict(X_test)
    print(accuracy_score(Y_test, Y_predict))
    
    print(pd.DataFrame(
    confusion_matrix(Y_test, Y_predict),
    columns=['L', 'W'],
    index=['True L', 'True W']
))

performDecTree(df1)